In [1]:
from keras.applications import MobileNet


img_rows, img_cols = 224, 224 
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))
for layer in MobileNet.layers:
    layer.trainable = False

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import PIL
from IPython.display import clear_output

In [2]:
#TO DETECT FACE
def detect_face(frame):   
    detector = cv2.CascadeClassifier("frontal_face.xml")
    faces = detector.detectMultiScale(frame,1.2,5)
    return faces

#Function to cut out the captured face
def cut_faces(image, faces_coord):
    faces = [] 
    for (x, y, w, h) in faces_coord:
        faces.append(image[y: y + h, x : x + w ])    
    return faces


#Function to resie the image
def resize(images,size=(47,62)):
    image_resize = []
    for image in images:
        if image.shape < size:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_CUBIC)
        else:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_AREA)
        image_resize.append(img_size)    
    return image_resize

#Function to show the captured pic
def plot_show(image,title=""):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()
    
#Function to put a rectangle in the face
def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        cv2.rectangle(image, (x , y), (x + w , y + h), (0,0,255),2)

In [ ]:
       
#CAPTURING THE IMAGES OF A PERSON/PERSONS for the dataset
count = int(input('Enter No. of Persons: '))

for i in range(1, count+1):
    cam = cv2.VideoCapture(0)
    #cv2.namedWindow("Face", cv2.WINDOW_AUTOSIZE)
    folder = "Storage/train/"+input(f'Person {i}:').lower()
    
    if not os.path.exists(folder):
        os.mkdir(folder)
        flag_start_capturing = False
        sample=1
        cv2.namedWindow("Face", cv2.WINDOW_NORMAL)

        while True:
            ret,frame = cam.read()
            faces_coord = detect_face(frame)

            if len(faces_coord):
                faces = cut_faces(frame, faces_coord)
                cv2.imwrite(folder + '/' + str(sample)+'.jpg',faces[0])
                plot_show(faces[0],"Capture Count:"+str(sample))
                clear_output(wait=True)
                if flag_start_capturing == True:
                    sample += 1
            
            draw_rectangle(frame,faces_coord)
            cv2.imshow('Face',frame)
            keypress=cv2.waitKey(1)
        
            if keypress == ord('c'):
                if flag_start_capturing == False:
                    flag_start_capturing = True
        
            if sample >100: # Capture Count
                break
            
            if keypress == ord('q'): # Quit
                break

        cam.release()
        cv2.destroyAllWindows()
    else:
        print ("This name already exists.")

In [5]:
def FChead(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = count

FC_Head = FChead(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Storage/train/'
validation_data_dir = 'Storage/validation/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 390 images belonging to 4 classes.
Found 93 images belonging to 4 classes.


In [7]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("FACE-CLASSIFY.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 390
nb_validation_samples = 93

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)


Epoch 1/5
24/24 [==============================] - 237s 10s/step - loss: 2.8720 - accuracy: 0.5545 - val_loss: 1.6991 - val_accuracy: 0.3822

Epoch 00001: val_loss improved from inf to 1.69910, saving model to FACE-CLASSIFY.h5
Epoch 2/5
24/24 [==============================] - 230s 10s/step - loss: 0.8537 - accuracy: 0.7249 - val_loss: 2.2939 - val_accuracy: 0.2597

Epoch 00002: val_loss did not improve from 1.69910
Epoch 3/5
24/24 [==============================] - 239s 10s/step - loss: 0.1895 - accuracy: 0.9420 - val_loss: 2.4688 - val_accuracy: 0.2662

Epoch 00003: val_loss did not improve from 1.69910
Epoch 4/5
24/24 [==============================] - 230s 10s/step - loss: 0.5106 - accuracy: 0.8994 - val_loss: 3.1473 - val_accuracy: 0.2484
Restoring model weights from the end of the best epoch

Epoch 00004: val_loss did not improve from 1.69910
Epoch 00004: early stopping


In [4]:
from keras.models import load_model
classifier = load_model('FACE-CLASSIFY.h5')

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [6]:

labels_dic = {}
people = [person for person in os.listdir("Storage/train/")]
for i, person in enumerate(people):
        labels_dic[i] = person

'akul'

In [7]:
cam = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)

while True:
    ret,frame = cam.read()
    
    faces_coord = detect_face(frame) # detect more than one face

    if len(faces_coord):
        faces = cut_faces(frame, faces_coord)
        for i, face in enumerate(faces): # for each detected face
            input_im = cv2.resize(face, (224, 224), interpolation = cv2.INTER_LINEAR)
            input_im = input_im / 255.
            input_im = input_im.reshape(1,224,224,3) 
            # Get Prediction
            res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
            name = labels_dic[res[0]].capitalize()
            if True:
                cv2.putText(frame, name, (faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
                
                  
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        break
        
cam.release()
cv2.destroyAllWindows()